# PyDMD

## User Manual 1: BOP-DMD on Flow Past a Cylinder Data

In [3]:
import numpy as np
import scipy.io as sio
import matplotlib.pyplot as plt

from pydmd import BOPDMD
from pydmd.plotter import plot_summary
from pydmd.preprocessing import zero_mean_preprocessing

### Import Data

In [4]:
# Import vorticity data and frame dimensions.
mat = sio.loadmat("CYLINDER_ALL.mat")

X = mat["VORTALL"]   # Vorticity data.
nx = mat["nx"][0][0] # Number of pixels along x-axis.
ny = mat["ny"][0][0] # Number of pixels along y-axis.
m = X.shape[-1]      # Number of time points.
t = np.arange(m)     # Time data.

# Add Gaussian noise to the data.
noise_magnitude = 0.2
rng = np.random.default_rng(seed=1234)
noise = noise_magnitude * rng.standard_normal(X.shape)
X += noise

print(X.shape)

FileNotFoundError: [Errno 2] No such file or directory: 'CYLINDER_ALL.mat'